In [1]:
# Local
from resources import Tools # type: ignore
from resources import FlowTools # type: ignore
import FeatureDetection
import OpticalFlow

# Math
import numpy as np
import matplotlib.pyplot as plt

# Data
import cv2 as cv

In [ ]:
# Get image from file
file = 'Source/Vary Clouds (South Horizon)/#9 2024-04-08 13.05.13.png'
Img1, Img2 = np.hsplit(cv.imread(file)[:,:,::-1], 2)
imgH, imgW = Img1.shape[:2]

p1L, p2L, c1, c2, translation, rotation, center, angle = FeatureDetection.FeatureDetection(Img1, Img2)
p0 = np.array([np.repeat(range(imgW), imgH), np.tile(range(imgH), imgW)])

In [ ]:
img, _, _, p1, p2 = Tools.mergeAndWarpImages(p0, p0, c1, c2, 2, translation, rotation, center, [Img1, Img2])

_, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8), constrained_layout=True)
ax1.imshow(img/np.max(img)); ax1.set_axis_off()

overlap = Tools.refineAndConcatenateOverlaps(p0, p1, p2, [Img1, Img2])
o1, o2 = np.hsplit(overlap, 2)

ax2.imshow((o1+o2)/np.max(o1+o2)); ax2.set_axis_off()

In [ ]:
cOptical1, cOptical2 = OpticalFlow.OpticalFlow(o1/np.max(o1), o2/np.max(o2), ransac_tol=5)

In [ ]:
c1o, c2o = c1+cOptical1/2, c2-cOptical2/2
img2, _, _, p1, p2 = Tools.mergeAndWarpImages(p0, p0, c1o, c2o, 2, translation, rotation, center, [Img1, Img2])
plt.figure(figsize=(20, 8)); plt.imshow(img2/np.max(img2)); plt.axis('off')

overlap = Tools.refineAndConcatenateOverlaps(p0, p1, p2, [Img1, Img2])
o1O, o2O = np.hsplit(overlap, 2)

_, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16), constrained_layout=True)
ax1.imshow((o1+o2)/np.max(o1+o2)); ax1.set_axis_off()
ax2.imshow(np.abs(o1-o2)/np.max(np.abs(o1-o2))); ax2.set_axis_off()
ax3.imshow((o1O+o2O)/np.max(o1O+o2O)); ax3.set_axis_off()
ax4.imshow(np.abs(o1O-o2O)/np.max(np.abs(o1O-o2O))); ax4.set_axis_off()

In [ ]:
i1, i2 = FlowTools.padImages(o1+o2, o1O+o2O)
plt.figure(figsize=(15, 15)); plt.imshow(np.abs(i1-i2)/np.max(np.abs(i1-i2))); plt.axis('off')